In [1]:
# 本代码由可视化策略环境自动生成 2021年5月8日12:36
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m2_run_bigquant_run(input_1, start_date,end_date):
    # 示例代码如下。在这里编写您的代码
    ins=input_1.read_pickle()['instruments']
    start_date=input_1.read_pickle()['start_date']
    end_date=input_1.read_pickle()['end_date']
    df=DataSource('bar1d_CN_FUND').read(ins,start_date=start_date,end_date=end_date)
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m2_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m4_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
#    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.set_commission(PerOrder(buy_cost=0.0001, sell_cost=0.0001, min_cost=0.02)) # 设置手续费

# 回测引擎：每日数据处理函数，每天执行一次
def m4_handle_data_bigquant_run(context, data):
    # 获取今日的日期
    today = data.current_dt.strftime('%Y-%m-%d')
    cash = context.portfolio.cash

    if today == '2020-06-29':
        instruments = ['513100.HOF']
        weights = [1]
        for i in range(len(instruments)):
            context.order_target_percent(instruments[i], weights[i])
    
    if today == '2021-04-07':
        instruments = ['513100.HOF']
        weights = [0.5]
        for i in range(len(instruments)):
            context.order_target_percent(instruments[i], weights[i])

# 回测引擎：准备数据，只执行一次
def m4_prepare_bigquant_run(context):
    # 加载预测数据
    
    df = context.options['data'].read_df()
    context.data=df
    pass
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m4_before_trading_start_bigquant_run(context, data):
    pass
# 策略容量计算函数
def m9_calculate_bigquant_run(results, volume_limit, frequency, output='display'):
    import numpy as np
    import pandas as pd
    from empyrical import alpha_beta_aligned, annual_return, annual_volatility, cum_returns, information_ratio, max_drawdown, sharpe_ratio

    risk_free_interest = 0.0001173

    def get_max_drawdown(returns):
        """计算最大回撤"""
        returns = returns.copy()
        df_cum = cum_returns(returns, 1.0)
        running_max = np.maximum.accumulate(df_cum)
        underwater = df_cum / running_max - 1
        valley = underwater.index[np.argmin(underwater)]
        peak = underwater[:valley][underwater[:valley] == 0].index[-1]
        try:
            recovery = underwater[valley:][underwater[valley:] == 0].index[0]
        except IndexError:
            recovery = np.nan
        return peak, valley, recovery

    def sum_buy_and_sell_turnover(transactions):
        """统计每日交易额"""
        total_turnover = 0.0
        for t in transactions:
            total = abs(t['amount'] * t['price'])
            commission = t['commission']
            total_turnover += total + commission
        return total_turnover

    def prepare_results(results):
        """处理传入的回测数据，计算出后续需要用到的数据"""
        # 策略累计收益率
        algorithm_cumulative_returns = results['algorithm_period_return'] + 1.0
        algorithm_cumulative_returns_shift = results['algorithm_period_return'].shift(1).fillna(0) + 1.0
        # 策略日收益率
        results['algorithm_simple_returns'] = algorithm_cumulative_returns / algorithm_cumulative_returns_shift - 1.0
        # 基准累计收益率
        benchmark_cumulative_returns = results['benchmark_period_return'] + 1.0
        benchmark_cumulative_returns_shift = results['benchmark_period_return'].shift(1).fillna(0) + 1.0
        # 基准日收益率
        results['benchmark_simple_returns'] = benchmark_cumulative_returns / benchmark_cumulative_returns_shift - 1.0
        # 超额收益
        results['excess_simple_returns'] = results['algorithm_simple_returns'] - results['benchmark_simple_returns']
        # 单利累计相对收益
        results['excess_cumulative_returns'] = results['excess_simple_returns'].cumsum()
        # 单利累计相对收益百分比
        results['excess_cumulative_ratio'] = results['excess_cumulative_returns']
        return results

    def get_stats(results):
        """计算收益相关信息"""
        excess_return_ratio = round(results['excess_cumulative_ratio'].iloc[-1] * 100, 2)
        annual_return_ratio = round(annual_return(results['excess_simple_returns']) * 100, 2)
        alpha, beta = alpha_beta_aligned(
            results['excess_simple_returns'],
            results['benchmark_simple_returns'],
            risk_free_interest
        )
        sharp_ratio = round(
            sharpe_ratio(
                results['excess_simple_returns'],
                risk_free_interest
            ), 2
        )
        ir = round(
            information_ratio(
                results['excess_simple_returns'],
                results['benchmark_simple_returns']
            ), 2
        )

        if sharp_ratio < -1.0e+6:
            sharp_ratio = 'n/a'

        # 收益波动率
        return_volatility = round(annual_volatility(results['excess_simple_returns']) * 100, 2)
        # 最大回撤和最大回撤天数
        max_drawdown_value = round(abs(max_drawdown(results['excess_simple_returns']) * 100), 2)
        peak, valley, recovery = get_max_drawdown(results['excess_simple_returns'])
        max_drawdown_days = abs((peak - valley).days)

        # 计算平均每日换手率
        daily_transaction = results['transactions'].apply(sum_buy_and_sell_turnover)
        daily_portfolio_value = results['portfolio_value'].shift(1)
        daily_turnover_ratio = daily_transaction / daily_portfolio_value
        average_daily_turnover = round(daily_turnover_ratio.mean() * 100, 2)

        ret = {
            'excess_return_ratio': excess_return_ratio,
            'annual_return_ratio': annual_return_ratio,
            'beta': round(beta, 2),
            'alpha': round(alpha, 2),
            'sharp_ratio': sharp_ratio,
            'ir': ir,
            'return_volatility': return_volatility,
            'max_drawdown': max_drawdown_value,
            'max_drawdown_days': max_drawdown_days,
            'average_daily_turnover': average_daily_turnover,
        }
        return ret

    def plot_excess_cumulative_ratio(results, output='script'):
        """组合单利相对收益净值作图"""
        options = {
            'legend': {
                'enabled': True,
            },
            'plotOptions': {
                'series': {
                    'showInNavigator': True,
                },
            },
            'series': [
                {
                    'name': '组合单利相对收益净值',
                    'type': 'spline',
                    'tooltip': {
                        'valueDecimals': 2,
                        'valueSuffix': ' ',
                    },
                },
            ],
            'yAxis': [
                {
                    'title': {
                        'text': '净值',
                    },
                    'labels': {
                        'format': '{value}'
                    },
                },
            ]
        }
        return T.plot(results['excess_cumulative_ratio'] + 1, options=options, output=output)

    def plot_strategy_capacity(results, volume_limit, frequency, output='script'):
        """计算策略容量并作图"""
        # 持仓详情
        positions = results['positions']
        # 获取回测开始时间和结束时间
        start_date = results.index.min().strftime('%Y-%m-%d')
        end_date = results.index.max().strftime('%Y-%m-%d')
        # 获取开盘 N 分钟行情数据
        frequency_cols_map = {
            '15m': ['wap_2_buy_volume', 'wap_2_twap_buy'],
            '30m': ['wap_3_buy_volume', 'wap_3_twap_buy'],
            '60m': ['wap_4_buy_volume', 'wap_4_twap_buy'],
        }
        rename_dict = {
            '15m': {
                'wap_2_buy_volume': 'volume',
                'wap_2_twap_buy': 'twap',
            },
            '30m': {
                'wap_3_buy_volume': 'volume',
                'wap_3_twap_buy': 'twap',
            },
            '60m': {
                'wap_4_buy_volume': 'volume',
                'wap_4_twap_buy': 'twap',
            },
        }
        bar_data = DataSource('bar1d_wap_CN_STOCK_A').read(start_date=start_date, end_date=end_date, fields=frequency_cols_map.get(frequency))
        bar_data.rename(columns=rename_dict.get(frequency), inplace=True)
        bar_data['amount'] = bar_data['volume'] * bar_data['twap']
        bar_data.set_index('date', inplace=True)
        # 时间序列
        date_range = np.unique(positions.index.drop_duplicates().strftime('%Y-%m-%d'))
        all_strategy_capacity = []
        for d in date_range:
            daily_positions = [p['sid'].symbol for p in positions[d][0]]
            if not daily_positions:
                # 当天无持仓则跳过，容量值按 0 处理
                all_strategy_capacity.append(0)
                continue
            daily_data = bar_data[bar_data.index == d]
            condition = daily_data['instrument'].isin(daily_positions) & (daily_data['volume'] != 0)
            daily_instrument = daily_data[condition].reset_index()[['instrument', 'amount']].to_dict('records')
            if not daily_instrument:
                # 未找到对应股票则跳过当天，容量值按 0 处理
                all_strategy_capacity.append(0)
                continue
            min_daily_instrument = min(daily_instrument, key=lambda x: x.get('amount'))
            min_daily_amount = min_daily_instrument.get('amount', 0)
            daily_strategy_capacity = min_daily_amount * volume_limit * len(daily_positions)
            all_strategy_capacity.append(daily_strategy_capacity)
        # 策略容量
        strategy_capacity_df = pd.DataFrame(data=all_strategy_capacity, columns=['strategy_capacity'],
                                            index=results.index)
        # 总资产
        strategy_capacity_df['portfolio_value'] = results['portfolio_value']
        options = {
            'title': {
                'text': ''
            },
            'subtitle': {
                'text': ''
            },
            'legend': {
                'enabled': True,
            },
            'plotOptions': {
                'series': {
                    'showInNavigator': True,
                },
            },
            'yAxis': [
                {
                    'labels': {
                        'format': '{value}',
                    },
                    'title': {
                        'text': '总资产/策略容量',
                    },
                }
            ],
            'series': [
                {
                    'name': '策略容量',
                    'type': 'column',
                    'yAxis': 0,
                    'tooltip': {
                        'valueSuffix': ' '
                    }
                },
                {
                    'name': '总资产',
                    'type': 'spline',
                    'color': '#ff3300',
                    'yAxis': 0,
                    'tooltip': {
                        'valueDecimals': 2,
                        'valueSuffix': ' '
                    }
                }
            ]
        }
        return T.plot(strategy_capacity_df, options=options, output=output)

    results = prepare_results(results)
    stats = get_stats(results)
    stats_html = f"""
    <div class='kpicontainer'>
        <ul class='kpi'>
            <li><span class='title'>超额收益率</span><span class='value'>{stats['excess_return_ratio']}%</span></li>
            <li><span class='title'>超额年化收益率</span><span class='value'>{stats['annual_return_ratio']}%</span></li>
            <li><span class='title'>阿尔法</span><span class='value'>{stats['alpha']}</span></li>
            <li><span class='title'>贝塔</span><span class='value'>{stats['beta']}</span></li>
            <li><span class='title'>夏普比率</span><span class='value'>{stats['sharp_ratio']}</span></li>
            <li><span class='title'>收益波动率</span><span class='value'>{stats['return_volatility']}%</span></li>
            <li><span class='title'>信息比率</span><span class='value'>{stats['ir']}</span></li>
            <li><span class='title'>最大回撤</span><span class='value'>{stats['max_drawdown']}%</span></li>
            <li><span class='title'>最大回撤天数</span><span class='value'>{stats['max_drawdown_days']}</span></li>
            <li><span class='title'>平均每日换手率</span><span class='value'>{stats['average_daily_turnover']}%</span></li>
        </ul>
    </div>
    {plot_excess_cumulative_ratio(results)}
    """
    strategy_capacity_html = plot_strategy_capacity(results, volume_limit=volume_limit, frequency=frequency)
    if output == 'script':
        return T.plot_tabs([
            ('单利超额收益概况', stats_html),
            ('策略容量', strategy_capacity_html),
        ], output='script', title='策略容量分析')
    T.plot_tabs([
        ('单利超额收益概况', stats_html),
        ('策略容量', strategy_capacity_html),
    ], title='策略容量分析')


m3 = M.input_features.v1(
    features="""
# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征

close_0
open_0
""",
    m_cached=False
)

m1 = M.instruments.v2(
    start_date='2020-06-29',
    end_date='',
    market='CN_FUND',
    instrument_list="""512700.HOF
515000.HOF
159902.ZOF
159920.ZOF
512400.HOF
515700.HOF
159938.ZOF
159915.ZOF
512580.HOF
512690.HOF
510330.HOF
512980.HOF
513500.HOF
512680.HOF
513100.HOF
512880.HOF
513050.HOF""",
    max_count=0
)

m2 = M.cached.v3(
    input_1=m1.data,
    run=m2_run_bigquant_run,
    post_run=m2_post_run_bigquant_run,
    input_ports='',
    params='{\'start\':\'2018-08-29\'}',
    output_ports=''
)

m5 = M.derived_feature_extractor.v3(
    input_data=m2.data_1,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m4 = M.trade.v4(
    instruments=m1.data,
    options_data=m5.data,
    history_ds=m5.data,
    start_date='2020-06-29',
    end_date='2021-05-08',
    initialize=m4_initialize_bigquant_run,
    handle_data=m4_handle_data_bigquant_run,
    prepare=m4_prepare_bigquant_run,
    before_trading_start=m4_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)

m7 = M.strategy_evaluate_risk_overview.v1(
    raw_perf=m4.raw_perf
)

m8 = M.strategy_interval_return.v1(
    raw_perf=m4.raw_perf
)

m9 = M.strategy_capacity_analyzer.v1(
    raw_perf=m4.raw_perf,
    volume_limit=0.025,
    frequency='15m',
    calculate=m9_calculate_bigquant_run
)

m11 = M.strategy_turn_analysis.v1(
    raw_perf=m4.raw_perf
)

m12 = M.strategy_top10_position_analysis.v1(
    backtest_ds=m4.raw_perf
)

m13 = M.strategy_average_position_analysis.v1(
    raw_perf=m4.raw_perf
)

m16 = M.strategy_brinson_analysis.v1(
    raw_perf=m4.raw_perf,
    benchmark_index='000300.HIX'
)


[2021-05-08 12:24:06.392077] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-05-08 12:24:06.438369] INFO: moduleinvoker: input_features.v1 运行完成[0.046276s].

[2021-05-08 12:24:06.444898] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-05-08 12:24:06.472515] INFO: moduleinvoker: instruments.v2 运行完成[0.027623s].

[2021-05-08 12:24:06.483312] INFO: moduleinvoker: cached.v3 开始运行..

...

[2021-05-08 12:26:22.107538] INFO: brinson归因分析: 数据准备完成...

[2021-05-08 12:26:31.279116] INFO: brinson归因分析: 单期brinson数据计算完成...

[2021-05-08 12:26:32.964585] INFO: brinson归因分析: 多期brinson数据计算完成...

nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,110000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,210000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,220000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,230000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,240000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,270000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,280000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,330000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,340000.0,0.0,0.0,0.0,0.0,0.0,0.0
nan,350000.0,0.0,0.0,0.0,0.0,0.0,0.0


[2021-05-08 12:26:33.178950] INFO: moduleinvoker: strategy_brinson_analysis.v1 运行完成[28.524821s].